In [1]:
%%file Q4b.py
import numpy as np
from  sympy import *
from collections import deque
from mpi4py import MPI
from  time import time
import math 
import sys 

"""
This code uses the 3-D parallelized adaptive algorithm to calculate the cell average.
tol is the tolerance.

result is the approximate result of the integration.
errorEstimate is the error estimated.
finestLevel is the finest grid in the interval.

Some part of the code could be referred to the following link. 
https://github.com/stefantaylor/Parallel-Programming-Languages-and-Systems/blob/master/cw2/aquadPartA.c

Divide the sector into two cells

Use the Jacobian transformation method to map an cubed-sphere into a referenced cube. 

Then, use the parallelized 3-D adaptive Simpson algorithm to get the integral.

A stack is used to store the sub-interval of the domain and sub-tolerance. It is initilized with [X1, X2, Y1, Y2, Z1, Z2, tol].

A farmer cpu is used to monitor the other worker cpu, which are used to calculate the integral or divide the interval.

The main idea of the algorithm is as follows:
1.  A worker cpu calculate the integral I1 and I2 by using Simpson rule and composite Simpson rule
2.  If the difference between them is smaller than the tolrance, it will return I2 to the farmer cpu.
    If not, it will divide the domain by 8, and send the interval to farmer cpu. This worker cpu will be idle.
3.  The farmer cpu pop a interval from the stack, and sends it to a worker cpu which is idle.
4.  After receiving the sub-interval from the worker cpus, the farmer cpu will add them to the stack.  

"""
# set KS1, KS2, ETA1, ETA2, R1, R2, tol, maxLevels, volume  in the following part
global KS1, KS2, ETA1, ETA2, R1, R2, tol, volume, maxLevels
R1   = 1.0
R2   = 10.0
KS1  = -np.pi/4
KS2  = np.pi/4
ETA1 = -np.pi/4
ETA2 = np.pi/4
tol = 0.05
maxLevels = 50000

# the volume is got from the Question 4a
volume = 348.71   
# tolerance of the integral
tol = tol*volume

comm = MPI.COMM_WORLD
rank = comm.rank
numprocs = comm.size

# the f function, inputs are r, ks, and eta
def f(r, ks, eta):
    x = r/sqrt(1+tan(ks)*tan(ks) +tan(eta)*tan(eta) )
    y =  r*tan(ks) /sqrt(1+tan(ks)*tan(ks) +tan(eta)*tan(eta) )
    z = r*tan(eta)/sqrt(1+tan(ks)*tan(ks) +tan(eta)*tan(eta) )
    return sin(sqrt(x**2 + y**2 + z**2))*f1(x,y,z)

def f1(x, y, z):
    if x <= 0.5*cos(np.pi*y):
        return 2.0 + f2(x-1.0/tan(sqrt(0.5*np.pi))*y)
    elif x > 0.5*cos(np.pi*y):
        return 2.0 + f2(x+1.0/tan(sqrt(0.5*np.pi))*y) + cos(2*np.pi*y)
    else:
        return 0.0
        
def f2(r):
    if r <= -1.0/3.0:
        return -r*sin(1.5*np.pi*r*r)
    elif abs(r) < 1.0/3.0:
        return abs(sin(2*np.pi*r))
    elif 1.0/3.0 <= r:
        return 2*r - 1 + sin(3*np.pi*r)/6.0
    else:
        return 0.0
    
# it returns the Jacobian matrix partial(x,y,z)/partial(r,ks,eta)
def Jacobian():
    r, ks, eta =  symbols('r ks eta', real=True)
    x = r/sqrt(1+tan(ks)*tan(ks) +tan(eta)*tan(eta) )
    y = r*tan(ks) /sqrt(1+tan(ks)*tan(ks) +tan(eta)*tan(eta) )
    z = r*tan(eta)/sqrt(1+tan(ks)*tan(ks) +tan(eta)*tan(eta) )
    Jaco =[[diff(x, r), diff(x, ks), diff(x, eta)], [diff(y, r), diff(y, ks), diff(y, eta)], [diff(z, r), diff(z, ks), diff(z, eta)]]
    
    return Jaco

# it returns the determinant of Jacobian matrix at the node of r0, ks0, eta0
def JacobianDeterm(Jaco, r0, ks0, eta0):    
    r, ks, eta =  symbols('r ks eta', real=True)
    JacoDeter = []
    for i in range(3):
        row =[]
        for j in range(3):
            row.append(float(Jaco[i][j].subs({r:r0, ks:ks0, eta:eta0 })))
        JacoDeter.append(row)
    Det = np.asarray(JacoDeter)
    return np.linalg.det(Det)

def farmer(numprocs,  maxLevels):
    myStack = deque()
    points = [ R1, R2,KS1, KS2, ETA1, ETA2, tol]
    myStack.append(points)
    worker = []
    newdata =[]
    status = MPI.Status()
    numberLevels = 0
    finestLevel = R2-R1
    for i in range(numprocs):
        worker.append(0)
    total = 0
    errorEstimate = 0
    
    while numberLevels <= maxLevels:   # while numberLevels <= maxLevels, do the loop
        numberLevels = 1+numberLevels
        for i in range(1, numprocs):
            # if the worker cpu is idle and the stack is not empty, pop it from the stack and send it to the worker cpu           
            if worker[i] == 0 and len(myStack) > 0:
                data = myStack.pop()
                worker[i] = 1;
                comm.send(data, dest = i, tag = 1 )
        # receive data from the worker cpus                        
        newdata = comm.recv(source = MPI.ANY_SOURCE, tag = MPI.ANY_TAG, status=status)
        source = status.Get_source()
        tag = status.Get_tag()
        
        # tag =1 means the interval is too big and the error of integration is larger than tolerance. 
        # we need to divide the interval by eight
        if tag == 1:
            numberLevels = numberLevels-1
            myStack.append([newdata[0], newdata[1], newdata[3], newdata[4], newdata[6], newdata[7], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[3], newdata[4], newdata[6], newdata[7], newdata[9]])
            myStack.append([newdata[0], newdata[1], newdata[4], newdata[5], newdata[6], newdata[7], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[4], newdata[5], newdata[6], newdata[7], newdata[9]])
            
            myStack.append([newdata[0], newdata[1], newdata[3], newdata[4], newdata[7], newdata[8], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[3], newdata[4], newdata[7], newdata[8], newdata[9]])
            myStack.append([newdata[0], newdata[1], newdata[4], newdata[5], newdata[7], newdata[8], newdata[9]])
            myStack.append([newdata[1], newdata[2], newdata[4], newdata[5], newdata[7], newdata[8], newdata[9]])

            if (newdata[1]-newdata[0]) < finestLevel:
                finestLevel = newdata[1]-newdata[0]
        # error of integration is smaller than tolerance. We can add the integral to the total
        else:
            total = total + newdata[0]
            errorEstimate = errorEstimate+ newdata[1]
        worker[source] = 0
        
        # flag=0, and stack is empty. The calculation will end
        flag = 0
        for i in range(1, numprocs):
            if worker[i] ==1:
                flag =1
                break
        if flag == 0  and len(myStack) == 0:
            break
    total = total/volume        
    # set all the tag=9, which means to end the calculation     
    for i in range(1, numprocs):
        comm.send(data, dest = i, tag = 9 )
    return total,  numberLevels, errorEstimate, finestLevel 

def worker(rank):
    status = MPI.Status()
    newdata = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    data = [0, 0, 0, 0, 0, 0, 0]
    while 1:
        data = comm.recv(source=0, tag= MPI.ANY_TAG, status=status) 
        tag = status.Get_tag()
        if tag == 9 :
            break
        else:
            r1 = data[0]
            r2 = data[1]
            ks1 = data[2]
            ks2 = data[3]
            eta1 = data[4]
            eta2 = data[5]
            tol = data[6]
            
            hr = data[1]-data[0]
            hks = data[3]-data[2]
            heta = data[5]-data[4]
            rmid = (r1+r2)/2
            ksmid = (ks1+ks2)/2
            etamid = (eta1+eta2)/2
            # calculate coarse grid    
            Jaco = Jacobian()
            wCoarse = np.array([1, 4, 1])
            nCoarse= wCoarse.size
            rGridCoarse = np.linspace(r1, r2, num=nCoarse)  
            ksGridCoarse = np.linspace(ks1, ks2, num=nCoarse)  
            etaGridCoarse = np.linspace(eta1, eta2, num=nCoarse)  
            I1 = 0
            for i in range(nCoarse):
                for j in range(nCoarse):
                    for k in range(nCoarse):
                        I1 = I1 + wCoarse[i]*wCoarse[j]*wCoarse[k]*f(rGridCoarse[k], ksGridCoarse[j], etaGridCoarse[i])*JacobianDeterm(Jaco, rGridCoarse[k], ksGridCoarse[j], etaGridCoarse[i])       
            I1 = I1*hr*hks*heta/216
            I1

             # calculate fine grid
            wFine = np.array([1, 4, 2, 4, 1])
            nFine = wFine.size
            rGridFine = np.linspace(r1, r2, num=nFine)  
            ksGridFine = np.linspace(ks1, ks2, num=nFine)  
            etaGridFine = np.linspace(eta1, eta2, num=nFine)  

            I2 = 0
            for i in range(nFine):
                for j in range(nFine):
                    for k in range(nFine):
                        I2 = I2 + wFine[i]*wFine[j]*wFine[k]*f(rGridFine[k], ksGridFine[j], etaGridFine[i])*JacobianDeterm(Jaco, rGridFine[k], ksGridFine[j], etaGridFine[i])
            I2 = I2*hr*hks*heta/216/8
            
            if abs(I2-I1 ) >= 15*tol :        
                newdata[0] = r1
                newdata[1] = rmid
                newdata[2] = r2
                newdata[3] = ks1
                newdata[4] = ksmid
                newdata[5] = ks2
                newdata[6] = eta1
                newdata[7] = etamid
                newdata[8] = eta2
                newdata[9] = tol/8
                comm.send(newdata, dest = 0, tag = 1 )
            else:
                newdata[0] = I2
                newdata[1] = (I2-I1)/15
                comm.send(newdata, dest = 0, tag = 2 )
    
if numprocs < 2:
    sys.exit("ERROR: Must have at least 2 processes to run") 

lrVolume = []
lrErrorEstimate = []
lrFinestLevel =[]
# lr =0 means left cell
# lr =1 means right cell
for lr in range(2):
    if rank == 0:
        if lr ==0:
            KS1  = -np.pi/4
            KS2  = 0.0            
            result, numberLevels, errorEstimate, finestLevel = farmer(numprocs, maxLevels)
            lrVolume.append(result)
            lrErrorEstimate.append(errorEstimate/volume)
            lrFinestLevel.append(finestLevel)
        if lr ==1:
            KS1  = 0.0
            KS2  = np.pi/4           
            result, numberLevels, errorEstimate, finestLevel = farmer(numprocs, maxLevels)
            lrVolume.append(result)
            lrErrorEstimate.append(errorEstimate/volume)
            lrFinestLevel.append(finestLevel)    
    
    else:
        worker(rank)

if rank == 0:
    print("############ Result of left cell ###############")
    print("volume of left:  " + str(lrVolume[0]))
    print("errorEstimate:   " + str(lrErrorEstimate[0]))
    print("finestLevel:     " + '1/'+  str((R2-R1)/lrFinestLevel[0]) + '*(R2-R1)')
    print("############ Result of right cell ###############")
    print("volume of right: " + str(lrVolume[1]))
    print("errorEstimate:   " + str(lrErrorEstimate[1]))
    print("finestLevel:     " + '1/'+  str((R2-R1)/lrFinestLevel[1]) + '*(R2-R1)')
    

Overwriting Q4b.py


In [2]:
!mpiexec -np 6 python Q4b.py

############ Result of left cell ###############
volume of left:  2.95019813355922
errorEstimate:   -0.0160691880731845
finestLevel:     1/8.0*(R2-R1)
############ Result of right cell ###############
volume of right: 3.78647505806275
errorEstimate:   -0.0207100749304543
finestLevel:     1/8.0*(R2-R1)
